In [16]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pickle

import matplotlib.pyplot as plt
import torch
import pandas as pd
import numpy as np
import random
import transformers
print(transformers.__version__)
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer
)
from transformers import BertTokenizer, BertForSequenceClassification

4.26.0


In [21]:
CACHE="/Users/jiayixian/cache" # /Users/jiayixian/.cache/huggingface/hub
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max.colwidth', 0)

data_path = ""
model_path = "/Users/jiayixian/cache/models--LightChen2333--joint-bert-slu-atis"
col_mapping = { # fix_name: col name
    'label': 'label',
    'text': 'text'
}

#### load data

In [18]:
"""df_data = pd.read_csv(data_path)
label_list = list(df_data[col_mapping['label']].unique())
label_list.sort()
num_labels = len(label_list)
print(num_labels, label_list)
label2id = { v: i for i, v in enumerate(label_list)}
id2label = { i: v for i, v in enumerate(label_list)}"""

"df_data = pd.read_csv(data_path)\nlabel_list = list(df_data[col_mapping['label']].unique())\nlabel_list.sort()\nnum_labels = len(label_list)\nprint(num_labels, label_list)\nlabel2id = { v: i for i, v in enumerate(label_list)}\nid2label = { i: v for i, v in enumerate(label_list)}"

#### load model

In [51]:
#model = AutoModelForSequenceClassification.from_pretrained('LightChen2333/joint-bert-slu-atis', cache_dir=CACHE)
config = AutoConfig.from_pretrained(model_path, cache_dir=CACHE)
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config, cache_dir=CACHE)
# transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

Some weights of the model checkpoint at /Users/jiayixian/cache/models--LightChen2333--joint-bert-slu-atis were not used when initializing BertForSequenceClassification: ['model.encoder._AutoEncoder__encoder.encoder.encoder.layer.11.attention.output.dense.bias', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.10.attention.output.LayerNorm.weight', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.9.attention.self.key.weight', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.3.output.LayerNorm.bias', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.6.attention.self.query.weight', 'model.encoder._AutoEncoder__encoder.encoder.embeddings.position_embeddings.weight', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.3.output.dense.weight', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.7.attention.output.dense.bias', 'model.encoder._AutoEncoder__encoder.encoder.encoder.layer.0.attention.self.key.weight', 'model.encoder._AutoEncoder

In [52]:
config

BertConfig {
  "_id2label": {
    "intent": [
      "atis_flight",
      "atis_airfare",
      "atis_airline",
      "atis_ground_service",
      "atis_quantity",
      "atis_city",
      "atis_flight#atis_airfare",
      "atis_abbreviation",
      "atis_aircraft",
      "atis_distance",
      "atis_ground_fare",
      "atis_capacity",
      "atis_flight_time",
      "atis_meal",
      "atis_aircraft#atis_flight#atis_flight_no",
      "atis_flight_no",
      "atis_restriction",
      "atis_airport",
      "atis_airline#atis_flight_no",
      "atis_cheapest",
      "atis_ground_service#atis_ground_fare"
    ],
    "slot": [
      "O",
      "B-fromloc.city_name",
      "B-toloc.city_name",
      "B-round_trip",
      "I-round_trip",
      "B-cost_relative",
      "B-fare_amount",
      "I-fare_amount",
      "B-arrive_date.month_name",
      "B-arrive_date.day_number",
      "I-fromloc.city_name",
      "B-stoploc.city_name",
      "B-arrive_time.time_relative",
      "B-arrive_time.tim

In [63]:

label2id = {v: i for i, v in enumerate(config._label2id["intent"])}
id2label = {i: v for v, i in label2id.items()}
num_labels = len(label2id)
config = AutoConfig.from_pretrained(model_path,
                                    num_labels=num_labels,
                                    label2id=label2id,
                                    id2label=id2label,
                                    cache_dir=CACHE)


In [ ]:
config._label2id["intent"]

In [68]:
with open(os.path.join(model_path, 'tokenizer.pkl'), 'rb') as handle:
    tokenizer = pickle.load(handle)
tokens = tokenizer.tokenize("i would like to find a flight from charlotte to las vegas that makes a stop in st. louis")
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)
labels = torch.tensor([num_labels]).unsqueeze(0)
outputs = model(input_ids=input_ids, labels=labels, return_dict=True)
print("Logits:", outputs.logits)



Input IDs: tensor([[1045, 2052, 2066, 2000, 2424, 1037, 3462, 2013, 5904, 2000, 5869, 7136,
         2008, 3084, 1037, 2644, 1999, 2358, 1012, 3434]])


IndexError: Target 21 is out of bounds.

In [33]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3241, -0.2763]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [30]:
type(model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [ ]:
output = model(**input_ids)

In [25]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [42]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([13]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

IndexError: Target 13 is out of bounds.

In [41]:
outputs

SequenceClassifierOutput(loss=tensor(0.8278, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.1355, -0.1178]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
"""config = AutoConfig.from_pretrained(path=model_path,
                                    num_labels=num_labels,
                                    labels2id=label2id,
                                    id2label=id2label,
                                    cache_dir=CACHE)"""
config = AutoConfig.from_pretrained(model_path, cache_dir=CACHE)
# hidden_dropout_prob = args.hidden_dropout_prob,
# attention_probs_dropout_prob = model_args.attention_probs_dropout_prob

#model = BertForSequenceClassification(config, no_cuda=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, config=config, cache_dir = CACHE, use_fast=True)


ValueError: Tokenizer class OpenSLUv1 does not exist or is not currently imported.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, config=config, cache_dir = CACHE, use_fast=True)

### LIME

`exp = explainer.explain_instance(a string of text, model.predict: output 1d ndarray of probs , num_features=6)`

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer

In [ ]:
explainer = LimeTextExplainer(class_names=class_names) # class_names: a list of strings